### Notebook - Data scrapping of players

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from bs4 import BeautifulSoup
import requests

##### 1. Parsing building

In [34]:
# Link OP.GG
url_test = 'https://www.leagueofgraphs.com/fr/summoner/euw/Cotchanad%C3%A9-1234'

# Parser building
r = requests.get(url_test)
soup = soup = BeautifulSoup(r.text, 'html.parser')
# Sauvegarde dans un fichier texte
with open('leagueofgraph.txt', 'w', encoding='utf-8') as f:
    f.write(soup.prettify())  # Utilise prettify() pour une indentation lisible


##### 2. General Information about the player

In [ ]:
# Player Name
title = soup.find('div', class_='txt')
name = title.text
print(name)
nickname = name.split(' ')[0]
print('The player\'s name is',nickname)

# WR
games_opgg = soup.find('span', class_='leading-[26px]')
games_opgg_txt = games_opgg.text.split()
wins = int(''.join(c for c in games_opgg_txt[0] if c.isdigit()))
looses = int(''.join(c for c in games_opgg_txt[1] if c.isdigit()))

games = wins + looses
winrate = round(wins/(games),2)
print('The player played ' + str(games) + ' games')
print('The player has a current winrate of ' + str(winrate*100) + '%')

# Rank
rank_opgg = soup.find('strong', class_='text-xl first-letter:uppercase')
rank = rank_opgg.text.strip()
print('The player current rank is',rank)


AttributeError: 'NoneType' object has no attribute 'text'

##### 3. Specific data about the player

In [27]:
# Specific data
url_data = 'https://op.gg/lol/summoners/euw/Cotchanad%C3%A9-1234/champions?queue_type=SOLORANKED'

# Parser building
r = requests.get(url_test)
soup = soup = BeautifulSoup(r.text, 'html.parser')

# Sauvegarde dans un fichier texte
with open('specific_data_opgg.txt', 'w', encoding='utf-8') as f:
    f.write(soup.prettify())  # Utilise prettify() pour une indentation lisible

In [24]:
# KDA
kda_opgg = soup.find('span', class_='hidden text-2xs md:inline')
kda_opgg_txt = kda_opgg.text.split()

AttributeError: 'NoneType' object has no attribute 'text'